In [64]:
import pandas as pd
from scipy.stats import boxcox
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import yaml
from utils import *
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('./data/data.csv', sep='|', header=None)
# show all columns
pd.set_option('display.max_columns', None)

# load config file containing the column names based on datatype.
config = load_config("./config.yml")

df.columns = ['query_id', 'veh_id', 'query_date', 'car_feature_1', 'car_feature_2', 'price', 'car_feature_3', 'car_feature_4', 'car_feature_5', 'cont_car_feature_1', 'cont_car_feature_2', 'cont_car_feature_3', 'car_feature_6', 'car_feature_7', 'car_feature_8', 'cont_car_feature_4', 'car_feature_9', 'car_feature_10', 'car_feature_11', 'query_feature_1', 'query_feature_2', 'duration', 'pickup_date', 'query_feature_3', 'target']

df[config['cat_cols']] = df[config['cat_cols']].astype('category')

df['duration'] = df['duration'].astype('int64')

df[config['cat_cols']] = df[config['cat_cols']].astype('category')

le = LabelEncoder()
df['car_feature_9'] = le.fit_transform(df['car_feature_9'])
df['query_feature_3'] = le.fit_transform(df['query_feature_3'])

In [65]:
class  preprocessing(object):
    
    def __init__(self, df):
        self.df = df

    def remove_outliers(self):
        ## remove outliers from the price column for target = 0 using IQR
        q1 = self.df[self.df['target'] == 0]['price'].quantile(0.25)
        q3 = self.df[self.df['target'] == 0]['price'].quantile(0.75)
        iqr = q3 - q1
        self.df = self.df[~((self.df['target'] == 0) & (self.df['price'] < (q1 - 1.5 * iqr)) | (self.df['price'] > (q3 + 1.5 * iqr)))]
        
        ## remove outliers from the price column for target = 1 using IQR
        q1_1 = self.df[ self.df['target'] == 1]['price'].quantile(0.25)
        q3_1 =  self.df[ self.df['target'] == 1]['price'].quantile(0.75)
        iqr_1 = q3_1 - q1_1
        self.df = self.df[~((self.df['target'] == 1) & (self.df['price'] < (q1_1 - 1.5 * iqr_1)) | (self.df['price'] > (q3_1 + 1.5 * iqr_1)))]
        return self.df

    def drop_duplicates(self):
        self.df = self.df.sort_values(by='price', ascending=False)
        self.df = self.df.drop_duplicates(subset=['price'])
        return self.df





In [66]:
df.columns

Index(['query_id', 'veh_id', 'query_date', 'car_feature_1', 'car_feature_2',
       'price', 'car_feature_3', 'car_feature_4', 'car_feature_5',
       'cont_car_feature_1', 'cont_car_feature_2', 'cont_car_feature_3',
       'car_feature_6', 'car_feature_7', 'car_feature_8', 'cont_car_feature_4',
       'car_feature_9', 'car_feature_10', 'car_feature_11', 'query_feature_1',
       'query_feature_2', 'duration', 'pickup_date', 'query_feature_3',
       'target'],
      dtype='object')

In [67]:
def calculate_time_difference(df: pd.DataFrame, col1: pd.Series, col2: pd.Series, column_name: str) -> pd.DataFrame:
    """
    __summary__: This function is used to calculate the time difference between two timestamps in days.

    parameters:
        df {pd.DataFrame} -- [dataframe]
        col1 {pd.Series} -- [first timestamp column]
        col2 {pd.Series} -- [second timestamp column]
        column_name {str} -- [name of the new column]
    returns:
        df {pd.DataFrame} -- [dataframe with new column]
    """
    df[col1] = pd.to_datetime(df[col1], errors='coerce')
    df[col2] = pd.to_datetime(df[col2], errors='coerce')
    df[column_name] = df[col1] - df[col2]
    df[column_name] = df[column_name].dt.days
    return df


In [68]:
import utils
class feat_engg(object):

    def __init__(self, df, config):
        self.df = df
        self.config = config
    
    def categorify_columns(self):
        """_summary_: A function to label encode the categorical columns using categorify function from utils.
        parameters:
            None
        returns:
            df {dataframe}: A dataframe with the label encoded columns

        """
        self.logger.info("Label encoding the categorical columns")

        for col in self.config["categorify_columns"]:
            self.df = categorify(df=self.df, cat=col, freq_treshhold=20)
        return self.df

    def count_encode_columns(self):
        self.logger.info("Count encoding the categorical columns")
        """_summary_: A function to take count (value_count) of the categorical columns using count_encode function from utils.
        parameters:
            None
        returns:
            df {dataframe}: A dataframe with the count encoded columns

        for col in self.config["count_encode_columns"]:
            self.df = count_encode(self.df, col)
        return self.df
        """
        for col in self.config["count_encode_columns"]:
            self.df = count_encode(self.df, col)
        return self.df
    
    def split_datetime_col(self):
        """_summary_: A function to split the datetime columns into year, month, day for the column 'pickup_date'.
        parameters:
            None
        returns:
            df {dataframe}: A dataframe with the split datetime columns
        """
        # Split datetime columns into year, month, day.
        for colname in self.df[self.config['date_cols']]:
            self.df[colname] = self.df[colname].astype('datetime64[ns]')
        # Using split_datetime function from utils.
        self.df = split_datetime(self.df, "pickup_date")
        return self.df
    
    def cal_time_diff(self):
        """_summary_: A function to calculate the time difference between two datetime columns.
        parameters:
            None
        returns:
            df {dataframe}: A dataframe with the time difference columns

        """
        # Calculate time difference between two datetime columns.
        self.df = calculate_time_difference(self.df, 'pickup_date',
                                'query_date', 'time_to_pickup')
        return self.df


In [69]:
pre = preprocessing(df)
df = pre.remove_outliers()
df = pre.drop_duplicates()


In [70]:
df_engg = feat_engg(df, config)
df_engg_ = df_engg.split_datetime_col()
df_engg_ = df_engg.cal_time_diff()

In [71]:
df_engg_.head()

,query_id,veh_id,query_date,car_feature_1,car_feature_2,price,car_feature_3,car_feature_4,car_feature_5,cont_car_feature_1,cont_car_feature_2,cont_car_feature_3,car_feature_6,car_feature_7,car_feature_8,cont_car_feature_4,car_feature_9,car_feature_10,car_feature_11,query_feature_1,query_feature_2,duration,pickup_date,query_feature_3,target,ts_weekday,ts_day,ts_month,time_to_pickup
593946,106173309007956303,123869990,2017-01-06 17:33:09,2,1,666.74,4,7,1,5,3,5,1,1,3,6.0,17,3,0,2,2,7,2017-07-04,0,0,1,4,7,178
138882,107105453090123607,738157749,2017-01-07 10:54:53,1,1,666.73,1,1,1,4,4,5,1,0,11,8.0,0,3,1,1,2,6,2017-04-08,0,0,5,8,4,90
302683,721484832599771,1650641730,2017-01-22 17:52:16,1,1,666.72,11,7,2,5,1,7,1,0,8,8.0,28,3,1,6,2,7,2017-04-11,0,0,1,11,4,78
790358,511484653596273,1455815187,2017-01-17 18:59:39,1,1,666.71,2,1,1,4,4,5,1,1,11,8.0,44,3,1,44,1,7,2017-02-12,1,0,6,12,2,25
777359,611484653973012,638296445,2017-01-17 20:11:03,1,1,666.70,11,7,1,5,6,5,1,0,4,8.0,28,3,1,2,2,7,2017-06-25,0,0,6,25,6,158
